In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DATETIME,Sequence
from sqlalchemy import Table, MetaData, ForeignKey,JSON,TEXT,Integer, String, MetaData, Float

from sqlalchemy.orm import sessionmaker
engine = create_engine('sqlite:///database.db', echo = True)


metadata = MetaData()

places = Table('places', metadata,
Column('id', Integer, primary_key=True),
Column('name', String),
Column('locality', String),
Column('postal_code', String),
Column('street_address', String),
Column('extra_info', String),
Column('image_url', String),
Column('description_intro', String),
Column('description_body', String),
Column('tags', String),
Column('lat', String),
Column('lon', String),
Column('category', String),
Column('Monday', JSON),
Column('Tuesday', JSON),
Column('Wednesday', JSON),
Column('Thursday', JSON),
Column('Friday', JSON),
Column('Saturday', JSON),
Column('Sunday', JSON),
Column('Exception', JSON), )


events = Table('events', metadata,
Column('id', String, primary_key=True),
Column('name', String),
Column('info_url', String),
Column('modified_at', String),
Column('tags', String),
Column('description_intro', String),
Column('lat', Float),
Column('lon', Float),
Column('event_starting_day', String),
Column('event_ending_day', String),
Column('image_url', String),
Column('address', String),
Column('postal_code', String),
Column('locality', String),
Column('additional_description', String),
Column('description_body', String),              
)



metadata.create_all(engine)

2021-07-29 11:31:59,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-29 11:31:59,290 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("places")
2021-07-29 11:31:59,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-29 11:31:59,296 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("places")
2021-07-29 11:31:59,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-29 11:31:59,299 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2021-07-29 11:31:59,301 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-29 11:31:59,303 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("events")
2021-07-29 11:31:59,305 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-29 11:31:59,307 INFO sqlalchemy.engine.Engine 
CREATE TABLE places (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	locality VARCHAR, 
	postal_code VARCHAR, 
	street_address VARCHAR, 
	extra_info VARCHAR, 
	image_url VARCHAR, 
	description_intro VARCHAR, 
	description_body VARCHAR, 
	tags VARCHAR, 
	lat VARCHAR

In [2]:
import json
import requests
import numpy as np
import pandas as pd
places_res = requests.get('http://open-api.myhelsinki.fi/v1/places/')
print(places_res.json().keys())
print(type(places_res.json().get('data')))
print(places_res.json().get('data')[0].keys())

dict_keys(['meta', 'data', 'tags'])
<class 'list'>
dict_keys(['id', 'name', 'source_type', 'info_url', 'modified_at', 'location', 'description', 'tags', 'opening_hours'])


In [3]:
places_res.json().get('data')[0]['opening_hours']

{'hours': [{'weekday_id': 1,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  {'weekday_id': 2,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  {'weekday_id': 3,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  {'weekday_id': 4,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  {'weekday_id': 5,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  {'weekday_id': 6,
   'opens': '10:00:00',
   'closes': '17:00:00',
   'open24h': False},
  {'weekday_id': 7, 'opens': None, 'closes': None, 'open24h': False}],
 'openinghours_exception': ''}

In [4]:
events_res = requests.get('http://open-api.myhelsinki.fi/v1/events/')
print(events_res.json().keys())
print(type(events_res.json().get('data')))
print(events_res.json().get('data')[0].keys())

dict_keys(['meta', 'data', 'tags'])
<class 'list'>
dict_keys(['id', 'name', 'source_type', 'info_url', 'modified_at', 'location', 'description', 'tags', 'event_dates'])


In [5]:
events_res.json().get('data')[0]['event_dates']

{'starting_day': '2021-12-29T13:00:00.000Z',
 'ending_day': '2021-12-29T18:00:00.000Z',
 'additional_description': None}

In [5]:
placesdata = [{'id':place.get('id'),
               'name': place.get('name').get('fi'),
             'locality':place.get('location').get('address').get('locality'),
             'postal_code':place.get('location').get('address').get('postal_code'),
             'street_address':place.get('location').get('address').get('street_address'),
             'extra_info':place.get('info_url'),             
             'image_url':place.get('description').get('images'),
            'tags':place.get('tags'),
            'description_intro': place.get('description').get('intro'),
            'description_body': place.get('description').get('body'),
             'lat':place.get('location').get('lat'),
             'lon':place.get('location').get('lon')} for place in places_res.json().get('data')]


df = pd.DataFrame(placesdata)

for k in range(len(df)):
    if df.image_url[k] is not None and len(df.image_url[k]) !=0:
        df.image_url[k]=df.image_url[k][0]['url']
    else:
        df.image_url[k] = ''
        
places_taglist = []
for k in range(len(df)):
    tags = ''
    for n in range(len(df.tags[k])):
        places_taglist.append(df.tags[k][n]['name'])
        tags = tags+df.tags[k][n]['name']+', '
    df.tags[k] = tags

df.loc[:,'category'] = np.nan

import re 
categories = ['Shopping', 'Accomodation & Hotel', 'Work & Study', 'Sight & Attracttion', 'Meeting places',
              'Restaurant & Cafe', 'Museum & Gallerties', 'Culture center', 'Banquet Venues',
             'Grocery', 'Sauna & Wellness', 'Meeting places & Auditorium','Nature & Sport',
             'Bar & Nightlife', 'Libraries', 'Architecture & Design']

for k,tag in enumerate(df.tags.tolist()):
    for i in range(len(categories)):
        if re.search(str(categories[i][:3]).lower(), str(tag).lower()) or re.search(str(categories[i][-5:]).lower(), str(tag).lower()):
            df.loc[k,'category'] = categories[i]

with open('opening_hours.txt') as json_file:
    opening_hours = json.load(json_file)


opendf = pd.DataFrame(opening_hours)

df = pd.concat([df,opendf[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Exception']]], axis = 1)

placesdata = df.to_dict(orient="records")
placesdata

C:\Users\Lien Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lien Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lien Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[{'id': '653',
  'name': 'A. Tillander',
  'locality': 'Helsinki',
  'postal_code': '00100',
  'street_address': 'Aleksanterinkatu 17',
  'extra_info': 'https://www.alexandertillander.fi/',
  'image_url': 'https://edit.myhelsinki.fi/sites/default/files/styles/api_1980x1020/public/2017-10/14657520_1627162420917017_2082522418259513419_n.jpg?h=1555b8b2&itok=cv8nhamq',
  'tags': 'SHOPPING, Jewellery, ',
  'description_intro': None,
  'description_body': 'Jalokiviseppä ja ateljee, myös koruaiheisia näyttelyitä.',
  'lat': 60.168922424316406,
  'lon': 24.94364356994629,
  'category': 'Shopping',
  'Monday': {'weekday_id': 1,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  'Tuesday': {'weekday_id': 2,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  'Wednesday': {'weekday_id': 3,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': False},
  'Thursday': {'weekday_id': 4,
   'opens': '10:00:00',
   'closes': '18:00:00',
   'open24h': F

In [6]:
eventsdf = pd.DataFrame(events_res.json().get('data'))
newCols = ['description_intro','description_intro', 'lat', 'lon', 'event_starting_day', 'image_url',
           'event_ending_day','address', 'postal_code', 'locality', ]
for col in newCols:
    eventsdf.loc[:, col] =np.nan

events_taglist =[]
for k in range(len(eventsdf)):
    eventsdf.loc[k, 'name']= eventsdf.loc[k, 'name'].get('fi')
    eventsdf.loc[k, 'description_intro'] =  eventsdf.loc[k,'description'].get('intro')
    eventsdf.loc[k, 'description_body'] =  eventsdf.loc[k,'description'].get('body')
    eventsdf.loc[k, 'lat'] =  eventsdf.loc[k,'location'].get('lat')
    eventsdf.loc[k, 'lon'] =  eventsdf.loc[k,'location'].get('lon')
    eventsdf.loc[k, 'address'] =  eventsdf.loc[k,'location'].get('address').get('street_address')
    eventsdf.loc[k, 'postal_code'] =  eventsdf.loc[k,'location'].get('address').get('postal_code')
    eventsdf.loc[k, 'locality'] =  eventsdf.loc[k,'location'].get('address').get('locality')
    eventsdf.loc[k, 'event_starting_day'] =  eventsdf.loc[k,'event_dates'].get('starting_day')
    eventsdf.loc[k, 'event_ending_day'] =  eventsdf.loc[k,'event_dates'].get('ending_day')
    eventsdf.loc[k, 'additional_description'] =  eventsdf.loc[k,'event_dates'].get('additional_description')

    if eventsdf.loc[k,'description']['images'] is not None and len(eventsdf.loc[k,'description']['images']) !=0:
        eventsdf.loc[k,'image_url'] = eventsdf.loc[k,'description']['images'][0].get('url')
    else:
        eventsdf.loc[k,'image_url'] =''

   
    tags = ''
    for n in range(len(eventsdf.tags[k])):
        tags = tags+eventsdf.tags[k][n]['name']+', '
        events_taglist.append(eventsdf.tags[k][n]['name'])
    eventsdf.tags[k] = tags

eventsdf=eventsdf[['id', 'name',  'modified_at', 
        'tags',  'description_intro', 'lat', 'lon',
       'event_starting_day', 'image_url', 'event_ending_day', 'address',
       'postal_code', 'locality', 'additional_description'
       ]]

eventsdata = eventsdf.to_dict(orient="records")
eventsdata

C:\Users\Lien Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[{'id': 'helmet:221489',
  'name': 'Pelataan bridgeä!',
  'modified_at': '2021-07-29T07:26:28.874Z',
  'tags': 'games, Senior citizens, Games, General, Vantaa, senior citizens, Other events, Events, ',
  'description_intro': 'Tule pelaamaan klassista korttipeliä mukavassa seurassa! Sopii sekä aloittelijoille että konkaripelaajille.',
  'lat': 60.26133728027344,
  'lon': 24.853647232055664,
  'event_starting_day': '2021-12-29T13:00:00.000Z',
  'image_url': 'https://www.helmet.fi/download/noname/{BE939459-A402-472D-AA46-9EF617CCAAB5}/83703',
  'event_ending_day': '2021-12-29T18:00:00.000Z',
  'address': 'Paalutori 3',
  'postal_code': '01600',
  'locality': 'Vantaa',
  'additional_description': nan},
 {'id': 'helmet:221177',
  'name': 'Tapulikaupungin kirjaston lukupiiri',
  'modified_at': '2021-07-29T07:26:28.874Z',
  'tags': 'Book clubs, Infonäytöt (Helsinki), General, conversation, Helsinki, reading, Events, ',
  'description_intro': 'Syksyn teema on tarinoita koulumaailmasta ja hiema

opening_hours=[]
for k in range(len(places_res.json().get('data'))):
    lst = places_res.json().get('data')[k].get('opening_hours').get('hours')
    if lst is None:
        opening_hours.append({'id':places_res.json().get('data')[k].get('id'),
                             'Monday':None,
                              'Tuesday':None, 'Wednesday':None, 'Thursday': None, 'Friday':None,
                              'Saturday': None, 'Sunday': None, 'Exception': None
                             })
    else:  
        opening_hours.append({'id':places_res.json().get('data')[k].get('id'),
                             'Monday':lst[0],
                              'Tuesday':lst[1], 'Wednesday':lst[2], 'Thursday': lst[3], 'Friday':lst[4],
                              'Saturday': lst[5], 'Sunday': lst[6], 'Exception': places_res.json().get('data')[k].get('opening_hours').get('openinghours_exception')
                             })

In [7]:
conn = engine.connect()
conn.execute(places.insert(),placesdata)

2021-07-29 11:36:16,209 INFO sqlalchemy.engine.Engine INSERT INTO places (id, name, locality, postal_code, street_address, extra_info, image_url, description_intro, description_body, tags, lat, lon, category, "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Exception") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-07-29 11:36:16,210 INFO sqlalchemy.engine.Engine [generated in 0.25105s] (('653', 'A. Tillander', 'Helsinki', '00100', 'Aleksanterinkatu 17', 'https://www.alexandertillander.fi/', 'https://edit.myhelsinki.fi/sites/default/files/styles/api_1980x1020/public/2017-10/14657520_1627162420917017_2082522418259513419_n.jpg?h=1555b8b2&itok=cv8nhamq', None, 'Jalokiviseppä ja ateljee, myös koruaiheisia näyttelyitä.', 'SHOPPING, Jewellery, ', 60.168922424316406, 24.94364356994629, 'Shopping', '{"weekday_id": 1, "opens": "10:00:00", "closes": "18:00:00", "open24h": false}', '{"weekday_id": 2, "opens": "10:00:00", "closes": "18:00:

2021-07-29 11:36:16,316 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
conn.execute(events.insert(),eventsdata)

2021-07-29 11:36:16,439 INFO sqlalchemy.engine.Engine INSERT INTO events (id, name, modified_at, tags, description_intro, lat, lon, event_starting_day, event_ending_day, image_url, address, postal_code, locality, additional_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-07-29 11:36:16,441 INFO sqlalchemy.engine.Engine [generated in 0.07610s] (('helmet:221489', 'Pelataan bridgeä!', '2021-07-29T07:26:28.874Z', 'games, Senior citizens, Games, General, Vantaa, senior citizens, Other events, Events, ', 'Tule pelaamaan klassista korttipeliä mukavassa seurassa! Sopii sekä aloittelijoille että konkaripelaajille.', 60.26133728027344, 24.853647232055664, '2021-12-29T13:00:00.000Z', '2021-12-29T18:00:00.000Z', 'https://www.helmet.fi/download/noname/{BE939459-A402-472D-AA46-9EF617CCAAB5}/83703', 'Paalutori 3', '01600', 'Vantaa', nan), ('helmet:221177', 'Tapulikaupungin kirjaston lukupiiri', '2021-07-29T07:26:28.874Z', 'Book clubs, Infonäytöt (Helsinki), General, conversation, 

In [9]:
from sqlalchemy.sql import select
s = select([places])
result =conn.execute(s)

for row in result:
    print(row['Friday'])

2021-07-29 11:36:16,524 INFO sqlalchemy.engine.Engine SELECT places.id, places.name, places.locality, places.postal_code, places.street_address, places.extra_info, places.image_url, places.description_intro, places.description_body, places.tags, places.lat, places.lon, places.category, places."Monday", places."Tuesday", places."Wednesday", places."Thursday", places."Friday", places."Saturday", places."Sunday", places."Exception" 
FROM places
2021-07-29 11:36:16,526 INFO sqlalchemy.engine.Engine [generated in 0.00238s] ()
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '17:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None

{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '15:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '16:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': True}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '14:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '14:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '06:15:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '17:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '15:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'ope

{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '08:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '17:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:30:00', 'closes': '14:30:00', 'open24h': False}
{'weekday_id': 5, 'opens': '07:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '15:30:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 

{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
None
{'weekday_id': 5, 'opens': '07:30:00', 'closes': '15:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
None
{'weekday_id': 5, 'opens': '16:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '20:30:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': True}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '05:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None,

{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '17:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '16:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '14:00:00', 'closes': '02:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'op

...

{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '15:00:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '17:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '22:23:00', 'open24h

{'weekday_id': 5, 'opens': '11:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '15:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '17:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '06:30:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:30:00', 'closes': '00:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '19:00:00', 'open24h': Fals

{'weekday_id': 5, 'opens': '10:00:00', 'closes': '19:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '08:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '08:00:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '07:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday

{'weekday_id': 5, 'opens': '10:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '17:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '18:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '12:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '09:00:00', 'closes': '02:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '21:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '16:00:00', 'closes': '00:00:00', 'open24h': False}
{'weekday_id': 5, 'op

{'weekday_id': 5, 'opens': '11:00:00', 'closes': '21:30:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '20:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '11:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': '10:00:00', 'closes': '18:30:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '18:00:00', 'closes': '22:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': '16:00:00', 'closes': '23:00:00', 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h': False}
{'weekday_id': 5, 'opens': None, 'closes': None, 'open24h

In [ ]:
result =conn.execute(s)
row = result.fetchone()